In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from xgboost import plot_importance

from hnmchallenge.constant import *
from hnmchallenge.data_reader import DataReader
from hnmchallenge.dataset import Dataset
from hnmchallenge.evaluation.python_evaluation import map_at_k, recall_at_k
from hnmchallenge.feature_manager import FeatureManager
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.stratified_dataset import StratifiedDataset

/home/aayush/.cache/pypoetry/virtualenvs/hnmchallenge-2TdcNkYi-py3.9/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
dataset = StratifiedDataset()
dr = DataReader()

In [3]:
base_load_path = dr.get_preprocessed_data_path() / "xgb_models"

In [4]:
MODEL_NAME = "xgb_v4.json"

In [5]:
model = xgb.XGBRanker()
model.load_model(base_load_path/MODEL_NAME)

In [6]:
model.best_ntree_limit

120

In [7]:
features = pd.read_feather(dr.get_preprocessed_data_path()/ "xgb_predictions_datasets" / "dataset_v4.feather")

In [8]:
customer_article_df = features[[DEFAULT_USER_COL, DEFAULT_ITEM_COL]].copy()
X = features.drop([DEFAULT_USER_COL, DEFAULT_ITEM_COL], axis=1)

In [9]:
X.head()

,ItemKNN_score,ItemKNN_rank,colour_group_code,department_no,popularity,popularity_last_month,number_bought,perceived_colour_master_id,perceived_colour_value_id,product_type_no,section_no,active,age,user_tendency
0,0.010266,1,71,1747,9558.0,93.0,25485,2,3,272,53,0.0,22.0,0.214286
1,0.010237,2,9,1747,18036.0,315.0,32251,5,4,272,53,0.0,22.0,0.214286
2,0.009981,3,73,1747,5006.0,142.0,16235,2,2,272,53,0.0,22.0,0.214286
3,0.006984,4,9,1722,2153.0,121.0,6142,5,4,272,15,0.0,22.0,0.214286
4,0.006967,5,10,1643,1229.0,NaN,5300,9,3,255,51,0.0,22.0,0.214286


In [10]:
y_pred = model.predict(X, ntree_limit=model.best_ntree_limit)

/home/aayush/.cache/pypoetry/virtualenvs/hnmchallenge-2TdcNkYi-py3.9/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/home/aayush/.cache/pypoetry/virtualenvs/hnmchallenge-2TdcNkYi-py3.9/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [11]:
customer_article_df["predicted_score"] = y_pred

In [12]:
sorted_scores = customer_article_df.sort_values([DEFAULT_USER_COL, "predicted_score"], ascending=False)

In [13]:
sorted_scores_index = sorted_scores.reset_index(drop=True)

In [14]:
sorted_scores_index 


,customer_id,article_id,predicted_score
0,1136205,7580,2.476382
1,1136205,15003,2.136454
2,1136205,1765,1.991270
3,1136205,838,1.856465
4,1136205,12955,1.731871
...,...,...,...
113620595,0,4018,-3.115914
113620596,0,2385,-3.121913
113620597,0,203,-3.408237
113620598,0,414,-3.422912


In [17]:
cutoff=[]
groups = sorted_scores_index.groupby(DEFAULT_USER_COL).size().values

In [19]:
cutoff=groups

In [22]:
cutoff

array([100, 100, 100, ..., 100, 100, 100])

In [21]:
abc = sorted_scores_index.groupby(DEFAULT_USER_COL)[DEFAULT_ITEM_COL].apply(list)

In [23]:
i=0
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=cut


In [24]:
final_df = sorted_scores_index.loc[filter_indices]

In [25]:
final_df

,customer_id,article_id,predicted_score
0,1136205,7580,2.476382
1,1136205,15003,2.136454
2,1136205,1765,1.991270
3,1136205,838,1.856465
4,1136205,12955,1.731871
...,...,...,...
107,1136204,21592,1.229490
108,1136204,16023,1.159974
109,1136204,14384,1.021737
110,1136204,12872,1.021459


In [ ]:
CUTOFF = sorted_scores_index.groupby(DEFAULT_USER_COL).size().values
filter_indices = []
for i in range(len(sorted_scores_index)):
    for j in CUTOFF:
        if i % j == 0:
            filter_indices.extend(range(i, i+12))